In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from env import github_token, github_username
import acquire as aqr
import prepare as prep
import explore as exp
import visual as vis
import model as m
import json
import nltk

import matplotlib.pyplot as plt
import numpy as np


[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [ ]:
# # Specify the file name where you want to save the list
# file_name = "repo_processed.json"

# # Load the REPOS list from the JSON file
# with open(file_name, 'r') as file:
#     repo_loaded = json.load(file)

In [12]:
repos_df = pd.read_csv("processed_repos_ngrams_v5.csv", index_col=0)

In [10]:
repos_df = pd.DataFrame(repo_loaded)
repos_df

,repo,language,readme
0,AtsushiSakai/PythonRobotics,Python,pythonrobotics githubactionlinuxci githubactio...
1,kiloreux/awesome-robotics,Other,awesome robotics awesome list various book cou...
2,NxRLab/ModernRobotics,Python,modern robotics mechanic planning control code...
3,mithi/robotics-coursework,Other,want use arduino raspberry pi make robot short...
4,onlytailei/CppRobotics,C++,cpprobotics cpp implementation pythonrobotics ...
...,...,...,...
670,RajashekarRaju/compose-actors,Other,compose actor dancer roadmap v030 x let user s...
671,MMehrez/MPC-and-MHE-implementation-in-MATLAB-u...,Other,failtoloadreadme
673,1c7/Translate-Subtitle-File,Other,srt vtt api key api 100 api 510 2023130 420 20...
674,landy22granatt/Kumpulan-Script-Termux,Python,pkg update pkg upgrade pkg install bash pkg in...


In [17]:
repos_df['text'] = repos_df['readme']
repos_df = repos_df.drop(columns=['readme'])
repos_df

,repo,language,bi-grams,text
0,AtsushiSakai/PythonRobotics,Python,path_planning grid_based path_tracking steerin...,pythonrobotics githubactionlinuxci githubactio...
2,NxRLab/ModernRobotics,Python,modern_robotics robotics_mechanic mechanic_pla...,modern robotics mechanic planning control code...
4,onlytailei/CppRobotics,C++,red_circle black_line steering_control point_r...,cpprobotics cpp implementation pythonrobotics ...
5,JdeRobot/RoboticsAcademy,Other,learn_robotics mini_radi roboticsacademy_learn...,roboticsacademy learn robotics artificial inte...
6,pptacher/probabilistic_robotics,C++,probabilistic_robotics victoria_park park_data...,probabilisticrobotics working detailed solutio...
...,...,...,...,...
665,Skythinker616/foc-wheel-legged-robot,Other,2000_2000 solidworks_matlab 15000_2000 foc_eng...,foc english solidworks matlab simulink simscap...
667,Sollimann/bonsai,Other,behavior_tree mut_bt bonsaibtsuccess_dt dt_els...,bonsai rust implementation behavior tree build...
669,zjunlp/EasyInstruct,Python,easyinstruct_import example_python python_easy...,easytouse instruction generation framework lar...
670,RajashekarRaju/compose-actors,Other,x_add detail_screen tmdb_api ui_state screen_x,compose actor dancer roadmap v030 x let user s...


In [18]:
len(repos_df)

543

In [20]:
extra_words = [
    'ro'
]

exclude_words = []

# List of other technologies, not languages
non_languages = [
    "CMake", "Jupyter Notebook", "HTML", "Go", "Shell", "TeX", 
    "CMake", "Dockerfile", "OpenSCAD", "MQL5", "Vue", "Smali", 
    "RobotFramework", "Cuda", "SCSS"
]

# Define the list of acceptable languages
labeled_languages = ['C++', 'Python']

# Process the DataFrame 'repos_df' by applying text preprocessing, filtering, and language categorization
repos_df = prep.process_dataframe(repos_df, extra_words, exclude_words, 'lemmatize', labeled_languages, non_languages)

# Display the first few rows of the processed DataFrame
repos_df.head()

# NOTES
# Remove any word bigger than n

KeyError: 'readme_contents'

In [9]:
len(repos_df)

541

In [10]:
train, val, test = m.nlp_train_val_test(repos_df, seed = 42)
train.shape, val.shape, test.shape

((378, 3), (81, 3), (82, 3))

In [58]:
word_counts = exp.word_counts(train, reset_index=True)


In [61]:
len(word_counts)

4058

In [59]:
chi2_results = m.chi2_test_for_all_words(word_counts)

In [45]:
# Display the first few rows of the results
chi2_results.head()

,word,p_value
0,robot,0.000007
1,paper,0.000000
2,install,0.000168
3,use,0.062587
4,using,0.000040


In [60]:
# Filter the words from chi2_results with p-values below 0.05
non_significant_words = chi2_results[chi2_results['p_value'] > 0.05]['word'].tolist()
len(non_significant_words)

0

In [16]:
with open("non_significant_words.txt", "w") as f:
    for word in non_significant_words:
        f.write("%s\n" % word)

In [17]:
with open("non_significant_words.txt", "r") as f:
    non_significant_words = [line.strip() for line in f]

In [47]:
len(extra_words)

24846

In [48]:
extra_words = extra_words + non_significant_words

In [49]:
len(extra_words)

24921

In [ ]:
# Define lists for additional and exclusionary words
extra_words = [
    'ro'
]
extra_words = extra_words + non_significant_words

exclude_words = []

In [50]:
train, val, test

train['text'] = train['text'].apply(lambda x: prep.remove_stopwords(x, extra_words, exclude_words))
val['text'] = val['text'].apply(lambda x: prep.remove_stopwords(x, extra_words, exclude_words))
test['text'] = test['text'].apply(lambda x: prep.remove_stopwords(x, extra_words, exclude_words))

In [51]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

In [75]:
#XGBOOST
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load and preprocess your data
train = pd.read_csv('train.csv', index_col=0)
val = pd.read_csv('val.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

train = train.dropna()
val = val.dropna()
test = test.dropna()

X_train = train.text
y_train = train.language

X_val = val.text
y_val = val.language

X_test = test.text
y_test = test.language

# Initialize and fit the label encoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

# Create TF-IDF vectors
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

# Create the XGBoost classifier instance
bst = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.25, objective='multi:softprob')

# Fit the XGBoost model on the training data
bst.fit(X_train_tfidf, y_train)

# Calculate accuracy scores for train, validation, and test sets
train_accuracy = accuracy_score(y_train, bst.predict(X_train_tfidf))
val_accuracy = accuracy_score(y_val, bst.predict(X_val_tfidf))
test_accuracy = accuracy_score(y_test, bst.predict(X_test_tfidf))

print(f'\nXGBClassifier Model (Hyperparameters Used)')
print(f'==================================================')
print(f'\nTrain Accuracy: {train_accuracy:.2f}\n')
print(f'Validation Accuracy: {val_accuracy:.2f}\n')
print(f'Test Accuracy: {test_accuracy:.2f}\n')



XGBClassifier Model (Hyperparameters Used)

Train Accuracy: 1.00

Validation Accuracy: 0.74

Test Accuracy: 0.76



In [79]:
m.johns_model_lr_1()


Logisitic Regression Model (Hyperparameters Used)

Train Accuracy: 0.97


Validation Accuracy: 0.78



In [80]:
m.johns_model_lr_test()


Final Model Logisitic Regression with Hyperparameter tuning

Train Accuracy: 0.97


Validation Accuracy: 0.78


Test Accuracy: 0.74



In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

def johns_model_lr_gs():
    # Load and preprocess your data
    train = pd.read_csv('train.csv', index_col=0)
    val = pd.read_csv('val.csv', index_col=0)
    
    train = train.dropna()
    val = val.dropna()
    
    X_train = train.text
    y_train = train.language
    X_val = val.text
    y_val = val.language

    # Create TF-IDF vectors
    tfidf = TfidfVectorizer()
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_val_tfidf = tfidf.transform(X_val)

    # Define the hyperparameter space
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [1, 1.5, 2, 2.5, 3],#, 3.5, 4, 4.5, 5, 7, 10],
        'fit_intercept': [True, False],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': [1,2,3,4,5,6,7,8,9,10, 25]
    }

    # Create a LogisticRegression model
    lm = LogisticRegression(random_state=42, class_weight='balanced')

    # Create the grid search object
    grid_search = GridSearchCV(lm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

    # Fit the grid search object to the data
    grid_search.fit(X_train_tfidf, y_train)

    # Get the best model from grid search
    best_lm = grid_search.best_estimator_

    # Calculate accuracy scores
    y_train_res = pd.DataFrame({'actual': y_train, 'preds': best_lm.predict(X_train_tfidf)})
    y_val_res = pd.DataFrame({'actual': y_val, 'preds': best_lm.predict(X_val_tfidf)})
    train_accuracy = accuracy_score(y_train_res['actual'], y_train_res['preds'])
    val_accuracy = accuracy_score(y_val_res['actual'], y_val_res['preds'])

    print(f'\nLogistic Regression Model (Best Hyperparameters)')
    print(f'==================================================')
    print(f'Best parameters found: {grid_search.best_params_}\n')
    print(f'Train Accuracy: {train_accuracy:.2f}\n')
    print(f'Validation Accuracy: {val_accuracy:.2f}\n')


In [68]:
johns_model_lr_gs()

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarni


Logistic Regression Model (Best Hyperparameters)
Best parameters found: {'C': 2, 'fit_intercept': True, 'max_iter': 10, 'penalty': 'l2', 'solver': 'liblinear'}

Train Accuracy: 0.97

Validation Accuracy: 0.78



/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
2640 fits failed out of a total of 8800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
880 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    retu

In [ ]:
johns_model_lr_gs()

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag


Logistic Regression Model (Best Hyperparameters)
Best parameters found: {'C': 2, 'fit_intercept': True, 'max_iter': 10, 'penalty': 'l2', 'solver': 'liblinear'}

Train Accuracy: 0.97

Validation Accuracy: 0.78

